### ספריות

In [3]:
import os
import sys
import fiona
import pandas as pd
import geopandas as gpd
from openpyxl import load_workbook

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### העלת משתנים להרצת הקוד

In [5]:
path = os.getcwd()

software_root_folder = os.path.dirname(path)

software_folder_location = r'{}\current'.format(software_root_folder)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_folder_location))

forecast_version_folder_location=df_inputs_outputs['location'][1]

output_location=df_inputs_outputs['location'][2]

sys.path.append(software_root_folder)

### פונקציות גלובליות

In [6]:
from global_functions import up_load_shp, up_load_df, drop_geo, make_point, up_load_gdb, change_cols_names

ממשיך את הקוד עכשיו שיכלול גם את המאפיינים הגיאוגרפים שעיסא צריך

In [7]:
col=['Taz_num','geometry']

In [8]:
taz_border=up_load_shp(r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic\current\background_files\TAZ_V4_230518_Published.shp')[col]

In [9]:
forecast_point = make_point(taz_border)

# Load data layers
DISTRICT = up_load_gdb(
r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_folder_location), 'TAZ_V3_2_220123_DISTRICT')
urban = up_load_gdb(
r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_folder_location), 'TAZ_V3_2_220123_urban')
SCHOOLDISTRICT = up_load_gdb(
r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_folder_location), 'TAZ_V3_2_220123_SCHOOLDISTRICT')
PUMA = up_load_gdb(
r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_folder_location), 'TAZ_V3_2_220123_PUMA')


# Geographical join between traffic zones and data layers
forecast_point_DISTRICT = forecast_point.sjoin(
DISTRICT)[['Taz_num', 'puma2040_csv_DISTRICT']]
forecast_point_urban = forecast_point.sjoin(
urban)[['Taz_num', 'BaseProjections2040_csv_urban']]
forecast_point_SCHOOLDISTRICT = forecast_point.sjoin(
SCHOOLDISTRICT)[['Taz_num', 'puma2040_csv_SCHOOLDISTRICT']]
forecast_point_PUMA = forecast_point.sjoin(
PUMA)[['Taz_num', 'puma2040_csv_PUMA']]


In [10]:

# Merge tables into one table
taz_border = (taz_border
        .merge(forecast_point_DISTRICT, on='Taz_num', how='left')
        .merge(forecast_point_urban, on='Taz_num', how='left')
        .merge(forecast_point_SCHOOLDISTRICT, on='Taz_num', how='left')
        .merge(forecast_point_PUMA, on='Taz_num', how='left'))

# Rename columns# Rename columns with flipped names
taz_border.rename(columns={'puma2040_csv_DISTRICT': 'DISTRICT'}, inplace=True)
taz_border.rename(columns={'BaseProjections2040_csv_urban': 'urban'}, inplace=True)
taz_border.rename(columns={'puma2040_csv_PUMA': 'PUMA'}, inplace=True)
taz_border.rename(columns={'puma2040_csv_SCHOOLDISTRICT': 'SCHOOLDISTRICT'}, inplace=True)



In [11]:
# Add constant columns
taz_border['REGION'] = 1
taz_border['slop'] = 0

להעלות את המידע שנעשה עד עכשיו כדי לייצא פורמט שעיסא רוצה

In [12]:
#העלה של נתוני אנשים התפלגות גילים שנוצר לפני הקוד הזה
path=r'{}\Intermediates'.format(software_folder_location)
taz=up_load_df(path,'taz_before_add_geo')

In [13]:
taz=taz.merge(drop_geo(taz_border),on='Taz_num',how='left')

In [14]:
taz['yosh']=0

taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

taz['jerusalem_city']=0

taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

taz=taz.reset_index()

פלט של ההיברדי

In [15]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [16]:

col_needed=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_city',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slop',
'urban']

col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slop',
'urban']

In [17]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [18]:
change_cols_names(taz, col_needed, col_new_name).to_excel(r'{}\BaseProjections2020_{}.xlsx'.format(output_location,file_date),index=False)

In [19]:

col_needed=['Taz_num','Agg_taz_nu','PUMA','DISTRICT','REGION','SCHOOLDISTRICT']

col_new_name=['TAZ','AGG_TAZ','PUMA','DISTRICT','REGION','SCHOOLDISTRICT']

In [20]:
change_cols_names(taz, col_needed, col_new_name).to_excel(r'{}\puma2020_{}.xlsx'.format(output_location,file_date),index=False)

פלט של הנוסף שצריך בשביל הABM

In [21]:
col_with_fix_num=['area',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'county',
 'majunivenr',
 'parktot',
 'superZone',
 'Taz1']

In [22]:
taz[col_with_fix_num]=1

In [23]:
taz['UNIVENRORTHFEMALE']=taz['Univ_UO']/2
taz['UNIVENRORTHMALE']=taz['Univ_UO']/2


In [24]:
taz['TOA1']=taz['SEA1']+taz['UOA1']+taz['ARA1']
taz['TOA2']=taz['SEA2']+taz['UOA2']+taz['ARA2']
taz['TOA3']=taz['SEA3']+taz['UOA3']+taz['ARA3']

In [25]:
taz['tazSector']=1 #ערבי

In [26]:
taz.loc[taz['main_secto']=='U_Orthodox','tazSector']=2

In [27]:
taz.loc[taz['main_secto']=='Jewish','tazSector']=3

In [28]:
taz.loc[taz['main_secto']=='Palestinian','tazSector']=4

In [29]:
forecast_point = make_point(taz_border)

# Load data layers
hibiz = up_load_shp(r'{}\background_files\highBusinessFlag.shp'.format(software_folder_location))[['HighBiz', 'geometry']]


In [30]:

forecast_point_hibiz = forecast_point.sjoin(hibiz)[['Taz_num', 'HighBiz']]

taz = taz.merge(forecast_point_hibiz, on='Taz_num', how='left')

taz.rename(columns={'HighBiz': 'highBusinessFlag'}, inplace=True)


In [31]:
def add_geo_info_gdb(taz,taz_border,software_folder_location,gdb_name,layer_name,col_name):

    forecast_point = make_point(taz_border)

    # Load data layers
    geo_info =  up_load_gdb(r'{}\background_files\{}.gdb'.format(software_folder_location,gdb_name), '{}'.format(layer_name))
    
    forecast_point_geo_info = forecast_point.sjoin(geo_info)[['Taz_num', '{}'.format(col_name)]]

    taz = taz.merge(forecast_point_geo_info, on='Taz_num', how='left')

    taz=taz.fillna(0)

    
    return taz

In [32]:
col_parking=['FreeBuffer',
'PaidBuffer',
'Rest_EmpBuffer',
'searchtime',
'walktime',
'cost']

In [33]:
for c in col_parking:
    taz=add_geo_info_gdb(taz,taz_border,software_folder_location,'parking_abm',c,c)

In [34]:
col_emp_EI=['ieold','EIProp']

In [35]:
for c in col_emp_EI:
    taz=add_geo_info_gdb(taz,taz_border,software_folder_location,'emp_EI',c,c)

In [37]:
taz['IEProp']=taz['ieold']

In [38]:
taz['perScaled']=1-taz['EIProp']

הייצוא עצמו לטובת הSED FILE

In [45]:
col_needed=['Taz_num',
 'Taz_num',
 'hh',
 'PUMA',
 'DISTRICT',
 'county',
 'area',
 'parktot',
 'majunivenr',
 'tazSector',
 'Indus',
 'Com_hotel',
 'Business',
 'Public',
 'emp_Education',
 'agri',
 'total_emp',
 'UOA1',
 'UOA2',
 'UOA3',
 'SEA1',
 'SEA2',
 'SEA3',
 'ARA1',
 'ARA2',
 'ARA3',
 'TOA1',
 'TOA2',
 'TOA3',
 'Univ_AR',
 'Univ_SE',
 'UNIVENRORTHMALE',
 'UNIVENRORTHFEMALE',
 'ieold',
 'superZone',
 'IEProp',
 'Taz1',
 'perScaled',
 'EIProp',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'PaidBuffer',
 'Rest_EmpBuffer',
 'FreeBuffer',
 'SCHOOLDISTRICT',
 'SCHOOLDISTRICT',
 'highBusinessFlag',
 'searchtime',
 'walktime',
 'cost']

In [47]:
col_new_name=['maz',
 'taz',
 'hh_total',
 'puma',
 'district',
 'county',
 'area',
 'parktot',
 'majunivenr',
 'tazSector',
 'Indus',
 'Com_hotel',
 'Off_Bsness',
 'Public',
 'Education',
 'Agri',
 'totemp',
 'UOA1',
 'UOA2',
 'UOA3',
 'SEA1',
 'SEA2',
 'SEA3',
 'ARA1',
 'ARA2',
 'ARA3',
 'TOA1',
 'TOA2',
 'TOA3',
 'UNIVENRARAB',
 'UNIVENRSEC',
 'UNIVENRORTHMALE',
 'UNIVENRORTHFEMALE',
 'ieold',
 'superZone',
 'IEProp',
 'Taz1',
 'perScaled',
 'EIProp',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'PaidBuffer',
 'Rest_EmpBuffer',
 'FreeBuffer',
 'schDistrict',
 'schDistrictAgg',
 'highBusinessFlag',
 'searchtime',
 'walktime',
 'cost']

In [48]:
change_cols_names(taz, col_needed, col_new_name).to_excel(r'{}\SED_2020_{}.xlsx'.format(output_location,file_date),index=False)

In [ ]:
#load excel file
workbook = load_workbook(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))

#open workbook
sheet = workbook.active

#modify the desired cell
sheet["B4"] = forecast_version_folder_location
sheet["B5"] = False
sheet["B6"] = r'{}\current\background_files\TAZ_V4_230518_Published.shp'.format(software_root_folder)

#save the file
workbook.save(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))